In [66]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 63.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 7.9 MB/s eta 0:00:00


In [67]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf

In [34]:
df = pd.read_stata('data/asia-industry_tidy.dta')

In [35]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
time,4950,2004-09-15 09:31:38.181818240,1991-01-01 00:00:00,1997-11-01 00:00:00,2004-09-16 00:00:00,2011-08-01 00:00:00,2018-06-01 00:00:00,NaN
year,4950.0,2004.254517,1991.0,1997.0,2004.0,2011.0,2018.0,7.941991
month,4950.0,6.445455,1.0,3.0,6.0,9.0,12.0,3.45197
cpi_sa,4476.0,83.251602,2.085514,65.621038,87.501161,103.419375,164.167303,31.1759
cpi_yoy_nsa,4452.0,5.758772,-58.334073,1.376713,3.158235,6.560935,166.795649,12.481521
exchange_rate_neer,2856.0,113.085975,68.560556,94.669304,102.233911,115.037263,439.44347,45.045363
exchange_rate_reer,3150.0,101.280872,31.77348,91.797276,99.589273,109.833138,151.87058,14.645276
exchnage_rate_vs_usd,4914.0,2338.366451,0.980258,2.675115,27.537891,1162.67595,22819.15625,4796.532269
ind_prod_const,2540.0,66465697330.87323,1369266370.84639,6370138806.592298,10709093525.67985,51457633014.483322,512076967919.937012,105896474112.986908
ind_prod_const_sa,2540.0,67428092413.154846,1598456033.62116,6121474046.621555,10728649947.156799,55205472524.094872,469327808285.255981,108104150121.514404


In [36]:
# Data Cleaning
df = df.loc[lambda x: x["year"] >= 1998]
df = df.drop(df.loc[lambda x: (x["year"] == 1998) & (x["month"] == 1)].index)
df = df.drop(df.loc[lambda x: (x["year"] == 2018) & (x["month"] > 4)].index)


In [37]:
df.groupby("countrycode").agg(
    mean=("ind_prod_const", "mean"),
    std_dev=("ind_prod_const", "first"),
    freq=("ind_prod_const", "count"),
).dropna().reset_index().sort_values(by="countrycode")

,countrycode,mean,std_dev,freq
0,CHN,2.046815e+11,4.956969e+10,243
1,IDN,2.729169e+10,1.591749e+10,243
2,MYS,8.150858e+09,4.526050e+09,243
3,PHL,7.193672e+09,6.599710e+09,243
4,SGP,4.840317e+09,2.324321e+09,243
5,THA,9.791123e+09,5.602920e+09,243
6,TWN,1.171419e+10,6.744322e+09,242
7,USA,2.640678e+11,2.344603e+11,243


In [12]:
df.head(1)

,time,year,month,country,countrycode,cpi_sa,cpi_yoy_nsa,exchange_rate_neer,exchange_rate_reer,exchnage_rate_vs_usd,ind_prod_const,ind_prod_const_sa,usa_imp_sa
1275,1998-02-01,1998.0,2.0,Philippines,PHL,53.158154,7.651601,132.896441,93.14446,40.14125,6.599710e+09,6.233136e+09,74184.0


### Feature engineering

In [38]:
usa_ind_prd = df[df['countrycode'] == 'USA']\
    .groupby('time')\
        ['ind_prod_const_sa'].max()\
            .reset_index()\
                .rename(columns={'ind_prod_const_sa':'usa_ip_sa'})

df = pd.merge(df, usa_ind_prd, on='time', how='left')
df['usa_ip_sa'] = np.where(df['usa_ip_sa']<0, 0, df['usa_ip_sa'])

In [43]:
chn_ind_prd = df[df['countrycode'] == 'CHN']\
    .groupby('time')\
        ['ind_prod_const_sa'].max()\
            .reset_index()\
                .rename(columns={'ind_prod_const_sa':'chn_ip_sa'})

df = pd.merge(df, chn_ind_prd, on='time', how='left')
df['chn_ip_sa'] = np.where(df['chn_ip_sa']<0, 0, df['chn_ip_sa'])

In [45]:
df["ln_ip"] = np.log(df["ind_prod_const_sa"] + 1)
df["ln_usa_ip"] = np.log(df["usa_ip_sa"] + 1)
df["ln_chn_ip"] = np.log(df["chn_ip_sa"] + 1)
df["ln_usa_imports"] = np.log(df["usa_imp_sa"] + 1)
df["ln_er_usd"] = np.log(df["exchnage_rate_vs_usd"] + 1)

In [47]:
df = df.dropna(subset=["ln_ip"])

In [49]:
df = df[df['countrycode'].isin(['MYS', 'PHL', 'SGP', 'THA'])]

In [50]:
df['countrycode'].value_counts()

countrycode
PHL    243
MYS    243
SGP    243
THA    243
Name: count, dtype: int64

In [53]:
# lagged variables
df2 = df.sort_values(by=["countrycode", "time"])
cols = ['ln_ip', 'ln_usa_ip', 'ln_chn_ip', 'ln_usa_imports', 'ln_er_usd']
for col in cols:
    df2[f"d{col}"] = df2.groupby("countrycode")[col].transform("diff")

In [62]:
# Time variables
df2['date'] = df2["time"].dt.to_period('M').astype(str)

In [63]:
lags_usa = " + ".join(["dln_usa_imports.shift({i})".format(i=i) for i in range(0, 5)])
lags_ip = " + ".join(["dln_ip.shift({i})".format(i=i) for i in range(1, 3)])
thai_formula = "dln_ip ~ " + lags_usa + " + " + lags_ip

In [74]:
model = smf.ols(thai_formula, df2[df2['countrycode'] == 'THA']).fit()

In [75]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 dln_ip   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     3.470
Date:                Sat, 15 Jun 2024   Prob (F-statistic):            0.00149
Time:                        18:56:47   Log-Likelihood:                 424.62
No. Observations:                 238   AIC:                            -833.2
Df Residuals:                     230   BIC:                            -805.5
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.0024      0.003      0.849      0.397      -0.003       0.008
dln_usa_imports.shift(0)     0.3430      0.121      2.846      0.005       0.106       0.580
dln_usa_imports.shift(1)     0.2805      0.122      2.301      0.022       0.040       0.521
dln_usa_imports.shift(2)     0.1851      0.115      1.603      0.110      -0.042       0.412
dln_usa_imports.shift(3)    -0.1226      0.120     -1.020      0.309      -0.359       0.114
dln_usa_imports.shift(4)    -0.1940      0.120     -1.620      0.107      -0.430       0.042
dln_ip.shift(1)             -0.1383      0.065     -2.142      0.033      -0.265      -0.011
dln_ip.shift(2)             -0.1630      0.064     -2.537      0.012      -0.290      -0.036
==============================================================================
Omnibus:                      130.425   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2412.659
Skew:                          -1.692   Prob(JB):                         0.00
Kurtosis:                      18.226   Cond. No.                         54.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [79]:
model.params.values[1:].sum()

0.1907954288513376